In [308]:
import numpy as np
import scipy 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.tree as tree
import sklearn.ensemble as ensemble

In [309]:
df = pd.read_csv("train.csv")
df.loc[df['SibSp'] > 5, 'SibSp'] = 5
df.loc[df['Parch'] > 5, 'Parch'] = 5
df['Embarked'] = df['Embarked'].fillna('S')
y = df['Survived']
del df['Survived']
del df['Name']
del df['Ticket']
del df['Cabin']
ides = df['PassengerId']
del df['PassengerId']
categorical_cols = ['Parch', 'SibSp', 'Pclass', 'Embarked', 'Sex']
df_cat = df[categorical_cols].astype('string')
del df['Pclass']
del df['Sex']
del df['SibSp']
del df['Parch']
del df['Embarked']
from sklearn.feature_extraction import DictVectorizer as DV
encoder = DV(sparse = False)
df_cat_coder = encoder.fit_transform(df_cat.T.to_dict().values())

In [310]:
def calculate_means(numeric_data):
    means = np.zeros(numeric_data.shape[1])
    for j in range(numeric_data.shape[1]):
        to_sum = numeric_data.iloc[:,j]
        indices = np.nonzero(~numeric_data.iloc[:,j].isnull())[0]
        correction = np.amax(to_sum[indices])
        to_sum /= correction
        for i in indices:
            means[j] += to_sum[i]
        means[j] /= indices.size
        means[j] *= correction
    return pd.Series(means, numeric_data.columns)
mns = calculate_means(df)
for i in df:
    df[str(i)] = df[str(i)].fillna(mns[str(i)])

In [311]:
X = np.hstack((df, df_cat_coder))
a = np.ones([X.shape[0],1])
X = np.hstack((X, a))


In [312]:
X.shape

(891L, 23L)

In [313]:
#for i in range(0, 23):
#    for j in range(i + 1, 23):
#        X_new = X.T[i] * X.T[j]
#       X_new = np.reshape(X_new, (891, 1))
#        X = np.hstack((X, X_new))

In [314]:
model_bagging = tree.DecisionTreeClassifier(max_depth = 4, max_features= 5)
les = ensemble.BaggingClassifier(base_estimator = model_bagging, n_estimators=500)

In [315]:
X

array([[ 22.        ,   7.25      ,   0.        , ...,   0.        ,
          0.        ,   1.        ],
       [ 38.        ,  71.2833    ,   1.        , ...,   0.        ,
          0.        ,   1.        ],
       [ 26.        ,   7.925     ,   0.        , ...,   0.        ,
          0.        ,   1.        ],
       ..., 
       [ 29.69911765,  23.45      ,   0.        , ...,   0.        ,
          0.        ,   1.        ],
       [ 26.        ,  30.        ,   1.        , ...,   0.        ,
          0.        ,   1.        ],
       [ 32.        ,   7.75      ,   0.        , ...,   0.        ,
          0.        ,   1.        ]])

In [316]:
les.fit(X, y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=500, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [317]:
result = les.predict(X)
result

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1,

In [318]:
df = pd.read_csv("test.csv")
df.loc[df['SibSp'] > 5, 'SibSp'] = 5
df.loc[df['Parch'] > 5, 'Parch'] = 5
del df['Name']
del df['Ticket']
del df['Cabin']
ides = df['PassengerId']
del df['PassengerId']
categorical_cols = ['Parch', 'SibSp', 'Pclass', 'Embarked', 'Sex']
df_cat = df[categorical_cols].astype('string')
del df['Pclass']
del df['Sex']
del df['SibSp']
del df['Parch']
del df['Embarked']
from sklearn.feature_extraction import DictVectorizer as DV
encoder = DV(sparse = False)
df_cat_coder = encoder.fit_transform(df_cat.T.to_dict().values())

In [319]:
def calculate_means(numeric_data):
    means = np.zeros(numeric_data.shape[1])
    for j in range(numeric_data.shape[1]):
        to_sum = numeric_data.iloc[:,j]
        indices = np.nonzero(~numeric_data.iloc[:,j].isnull())[0]
        correction = np.amax(to_sum[indices])
        to_sum /= correction
        for i in indices:
            means[j] += to_sum[i]
        means[j] /= indices.size
        means[j] *= correction
    return pd.Series(means, numeric_data.columns)
mns = calculate_means(df)
for i in df:
    df[str(i)] = df[str(i)].fillna(mns[str(i)])

In [320]:
X = np.hstack((df, df_cat_coder))
a = np.ones([X.shape[0],1])
X = np.hstack((X, a))

In [321]:
#for i in range(0, 23):
#    for j in range(i + 1, 23):
#        X_new = X.T[i] * X.T[j]
 #       X_new = np.reshape(X_new, (418, 1))
 #       X = np.hstack((X, X_new))

In [322]:
df_cat_coder.shape

(418L, 20L)

In [323]:
X.shape

(418L, 23L)

In [324]:
result = les.predict(X)

In [325]:
result

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [326]:
output = pd.DataFrame({'PassengerId': ides , 'Survived': result})
output.to_csv('submission.csv', index=False)